In [1]:
from q_value_calc_crosslinks import calcQ
from idXML2df_all_hits import readAndProcessIdXMLAllHits
from functions import get_target_id, read_top_down_data, get_datasets, rerank
import re

In [2]:
i = 5
dataset = get_datasets()

In [3]:
input_files = [f"../data/{dataset[i]['type']}/{dataset[i]['file']}{dataset[i]['file_ending']}"]
if dataset[i]['comparison'] != None: 
      input_files.append(f"../data/{dataset[i]['type']}/{dataset[i]['file']}_{dataset[i]['comparison']}{dataset[i]['file_ending']}")

In [4]:
for input_file in input_files:
    if dataset[i]['type'] == 'crosslink_data':
        df = readAndProcessIdXMLAllHits(input_file)
        classColName = 'NuXL:isXL'
    elif dataset[i]['type'] == 'top_down_data':
        df = read_top_down_data(input_file)
        display(df)
        df = rerank(df, 'Scan', 'Score', 'PSMId')
        classColName = 'ModCount'
    file_ending = dataset[i]['file_ending']
    output_file = re.sub(file_ending, '.pkl', input_file)
    df.to_pickle(output_file)
    # filter dataframes
    filter_col = 'PSMId'
    filter_val = 1
    if dataset[i]['type'] == 'crosslink_data':
        df = df.loc[df[filter_col] == filter_val,:]
    # compute q-values
    df.sort_values(['Score', 'Label'], ascending=[False, True], inplace=True)
    q_vals = calcQ(df, classColName = classColName)
    # compute target IDs
    target_id = get_target_id(q_vals)
    output_file = re.sub(file_ending, '_filtered.pkl', input_file)
    target_id.to_pickle(output_file)

,ProteoformIndex,Scan,RetentionTime,NumMass,ProteinAccession,ProteinDescription,PrecursorMass,ProteoformMass,DatabaseSequence,ProteinSequence,...,ModMass,ModID,ModAccession,ModStart,ModEnd,PrecursorQscore,Score,PrSMLevelQvalue,ProteoformLevelQvalue,Label
0,0,968,453.432560,9,DECOY_sp|P77260|YDFI_ECOLI,Uncharacterized oxidoreductase YdfI OS=Escheri...,5192.565498,5192.565498,GEDYKCRSQLAGGQVAATVVQSQAVSTNLMVPIQPTDFLGHQVLPK...,KDILTNAAWTSTVHSHFYGVHDRAKDRLASLPAIYAHVRKTSA,...,389.054487,"Biotin:Thermo-21328,","1841,",446,485,0.91032,4,0.287037,0.359726,0.0
1,1,968,453.432560,9,sp|P77754|SPY_ECOLI,Periplasmic chaperone Spy OS=Escherichia coli ...,5192.565498,5192.565498,MRKLTALFVASTLALGAANLAHAADTTTAAPADAKPMMHHKGKFGP...,KDLNLTDAQKQQIREIMKGQRDQMKRPPLEERRAMHDIIAS,...,306.011283,"PhosphoUridine,","417,",56,93,0.91032,4,0.287037,0.359726,1.0
2,2,968,453.432560,9,DECOY_sp|P33346|YEHI_ECOLI,Uncharacterized protein YehI OS=Escherichia co...,5192.565498,5192.565498,VTVELILMIEAEWKAARVPEGKPAELGLEELLRAQLWYLGALQLAP...,KDLTVKRHASLTRWIAGGTEEGSKPGELRPALPEDCLTRGSAAKYL,...,214.940776,"SPITC,IDEnT,","261,762,",588,630,0.91032,4,0.287037,0.359726,0.0
3,3,968,453.432560,9,sp|P23909|MUTS_ECOLI,DNA mismatch repair protein MutS OS=Escherichi...,5192.565498,5192.565498,MSAIENFDAHTPMMQQYLRLKAQHPEILLFYRMGDFYELFYDDAKR...,KVEIGPIDRIFTRVGAADDLASGRSTFMVEMTETANILHNATEYSL,...,154.048625,"Glycerophospho,Delta:H(6)C(7)O(4),UgiJoulliePr...","419,1929,1282,721,",647,689,0.91032,8,0.268200,0.343805,1.0
4,4,968,453.432560,9,DECOY_sp|P23538|PPSA_ECOLI,Phosphoenolpyruvate synthase OS=Escherichia co...,5192.565498,5192.565498,LSMVDDQVIKNAGITFFPEKVPPRLNVFGMANGALSAKTQLKDFVT...,KDIWDIQPRLAGSKVTIEIIGGTTFPETSRESDYGPQFVIIRGATY,...,67.909431,"dichlorination,","937,",581,623,0.91032,4,0.287037,0.359726,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57016,57016,14646,5399.224109,36,sp|P52108|RSTA_ECOLI,Transcriptional regulatory protein RstA OS=Esc...,-1.000000,-1.000000,MNTIVFVEDDAEVGSLIAAYLAKHDMQVTVEPRGDQAEETILRENP...,MNTIVFVEDDAEVGSLIAAYLAKHDMQVTVEPRGDQAEETILRENP...,...,94.052461,"Delta:H(6)C(6)O(1),","205,",120,133,0.00000,16,0.465485,0.496234,1.0
57017,57017,14646,5399.224109,36,sp|P0AGC3|SLT_ECOLI,Soluble lytic murein transglycosylase OS=Esche...,-1.000000,-1.000000,MEKAKQVTWRLLAAGVCLLTVSSVARADSLDEQRSRYAQIKQAWDN...,ENARLMIPSLAQAQQLNEDQIQELRDIVAWRLMGNDVTDEQAKWRD...,...,-21.793639,NaN,NaN,258,283,0.00000,8,0.490398,0.496821,1.0
57018,57018,14646,5399.224109,36,sp|P0ACD8|MBHL_ECOLI,Hydrogenase-1 large chain OS=Escherichia coli ...,-1.000000,-1.000000,MSTQYETQGYTINNAGRRLVVDPITRIEGHMRCEVNINDQNVITNA...,MSTQYETQGYTINNAGRRLVVDPITRIEGHMRCEVNINDQNVITNA...,...,-11.704807,NaN,NaN,328,353,0.00000,8,0.490398,0.496821,1.0
57019,57019,14646,5399.224109,36,DECOY_sp|P0DP64|YBEM_ECOLI,Putative protein YbeM OS=Escherichia coli (str...,-1.000000,-1.000000,QATCGEHLHAIEVNGKRAEGWAQLEFEVELTDDLLVEVVPPMLAYG...,QATCGEHLHAIEVNGKRAEGWAQLEFEVELTDDLLVEVVPPMLAYG...,...,22.141180,NaN,NaN,59,86,0.00000,1,0.495504,0.504719,0.0


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


In [9]:
import pandas as pd
import numpy as np
input_file_original = f"../data/{dataset[i]['type']}/{dataset[i]['file']}_filtered.pkl"
df = pd.read_pickle(input_file_original)
df.loc[df['class-specific_q-val'] > 0,['ModCount', 'Label','FP', 'Count', 'class-specific_FDR', 'class-specific_q-val', 'Score', 'PrSMLevelQvalue' ]].head(50)

,ModCount,Label,FP,Count,class-specific_FDR,class-specific_q-val,Score,PrSMLevelQvalue
40365,2,1.0,1.0,200,0.005000,0.004673,74,0.000000
41772,2,0.0,1.0,200,0.005000,0.004673,74,0.004762
14030,2,1.0,1.0,205,0.004878,0.004673,73,0.004762
13859,2,1.0,1.0,205,0.004878,0.004673,73,0.004762
31031,2,1.0,1.0,205,0.004878,0.004673,73,0.004762
13471,2,1.0,1.0,205,0.004878,0.004673,73,0.004762
48193,2,1.0,1.0,205,0.004878,0.004673,73,0.004762
30201,2,1.0,1.0,208,0.004808,0.004673,72,0.004762
30081,2,1.0,1.0,208,0.004808,0.004673,72,0.004762
28204,2,1.0,1.0,208,0.004808,0.004673,72,0.004762
